In [133]:
import numpy as np
import pandas as pd
import xarray as xr
import os
from netCDF4 import Dataset

### Download latest version of params file from google drive
* requires 'publishing' the google drive spreadsheet
* file > publish to web
* then it can be set up to continuously publish the spreadsheet to a stable url (with some latency, maybe 1-2 minutes)

In [2]:
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
cmd = 'curl '+data_url+' > params.csv'
os.system(cmd)

0

### Read in csv data, filtering by the "include" column

In [36]:
data     = pd.read_csv('params.csv',header=0,skiprows=[1])
included = data['include']==1
params   = data.loc[included,['name','min','max','location','pft_mins','pft_maxs']]

np.fromstring(params['pft_mins'][2],dtype='float',sep=',')


params

,name,min,max,location,pft_mins,pft_maxs
1,displar,0.4,0.95,P,NaN,NaN
2,dleaf,pft,pft,P,"0.000216,0.000216,0.00072,0.0081,0.0081,0.0081...","0.00108,0.00108,0.0036,0.0567,0.0567,0.243,0.2..."
4,kmax,1.00E-08,3.00E-08,P,NaN,NaN
12,baseflow_scalar,0.0005,0.1,N,NaN,NaN
13,maximum_leaf_wetted_fraction,0.01,0.5,N,NaN,NaN


### Read in default parameter file
 * file may change as Keith pulls out more hard-coded params

In [220]:
filepath = '/glade/p/cgd/tss/people/oleson/modify_param/clm5_params.c200402.nc'
ds = xr.open_dataset(filepath)

npft = ds['pft'].values.size
pftnames = []
for i in np.arange(npft):
    x = str(ds['pftname'][i].values)[2:-1].strip()
    pftnames.append(x)

pftnames[:15]

['not_vegetated',
 'needleleaf_evergreen_temperate_tree',
 'needleleaf_evergreen_boreal_tree',
 'needleleaf_deciduous_boreal_tree',
 'broadleaf_evergreen_tropical_tree',
 'broadleaf_evergreen_temperate_tree',
 'broadleaf_deciduous_tropical_tree',
 'broadleaf_deciduous_temperate_tree',
 'broadleaf_deciduous_boreal_tree',
 'broadleaf_evergreen_shrub',
 'broadleaf_deciduous_temperate_shrub',
 'broadleaf_deciduous_boreal_shrub',
 'c3_arctic_grass',
 'c3_non-arctic_grass',
 'c4_grass']

## Generate parameter files
* ### this will overwrite parameter files!!
* ### proceed with caution

In [254]:
nparams = len(list(ds.data_vars))
ongoogle = np.zeros(nparams)
patterns = []
for p in data['name']:
    if p in ds.data_vars:
        shp = ds[p].values.shape
        if ds[p].values.size<79:
            v = ds[p].values+np.zeros(14)
        elif shp[0]<79:
            v = ds[p].values[0,1:15]
        else:
            v = ds[p].values[1:15]
            
        u = np.unique(v)
        pattern = np.zeros(14)
        for i in np.arange(14):
            pattern[i] = list(u).index(v[i])
        if len(patterns)==0:
            patterns.append(pattern)
        else:
            found = 0
            for i in np.arange(len(patterns)):
                if np.array_equal(pattern, patterns[i]):
                    found = 1
            if not found:
                patterns.append(pattern)

patterns = [patterns[i] for i in [1,6]]
patterns
#pattern0 = all equal
#pattern4 = NT,BT,S,G
#otherwise .... varies



[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([4., 1., 1., 5., 3., 5., 3., 1., 2., 2., 0., 0., 1., 1.])]

In [344]:

nparams = len(list(ds.data_vars))
ongoogle = np.zeros(nparams)
ct = -1
longname = []
units =[]
default=[]
for p in data['name']:
    if p in ds.data_vars:
        longname.append(ds[p].long_name)
        units.append(ds[p].units)
        ct = ct+1
        shp = ds[p].values.shape
        if ds[p].values.size<79:
            v = ds[p].values+np.zeros(14)
        elif shp[0]<79:
            v = ds[p].values[0,1:15]
        else:
            v = ds[p].values[1:15]

        if np.all(abs(v-np.mean(v))<0.0001):
            val = v[0]
            if (abs(val)>=1000)|(abs(val)<0.01):
                x = f"{Decimal(val):.2E}"
            else:
                x = str(val)
            default.append(x)
            pftvals.append('all pfts equal')
        else:
            themin = min(v)
            themax = max(v)
            if (abs(themin)>=1000)|(abs(themin)<0.01):
                x = f"{Decimal(themin):.2E}"
                y = f"{Decimal(themax):.2E}"
            else:
                x = str(np.around(themin,4))
                y = str(np.around(themax,4))
            default.append('varies: '+x+' to '+y)
    else:
        longname.append('no exact match in params file')
        units.append('tbd')
        default.append('tbd')
d = {'name':data['name'],'long_name':longname,'default value':default,'units':units}
df = pd.DataFrame(data=d)
df.to_csv('google_drive_sheet1_fillin.csv')

### Generate namelist files

In [231]:
ds['q10_mr'].values.size

1

In [289]:
from decimal import Decimal
f"{Decimal('40800000000.00000000000000'):.2E}"

'4.08E+10'